In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [24]:
#df = pd.read_csv('yellow_tripdata_2018-01.csv')
df = pd.read_csv('green_tripdata_2018-01.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793529 entries, 0 to 793528
Data columns (total 19 columns):
VendorID                 793529 non-null int64
lpep_pickup_datetime     793529 non-null object
lpep_dropoff_datetime    793529 non-null object
store_and_fwd_flag       793529 non-null object
RatecodeID               793529 non-null int64
PULocationID             793529 non-null int64
DOLocationID             793529 non-null int64
passenger_count          793529 non-null int64
trip_distance            793529 non-null float64
fare_amount              793529 non-null float64
extra                    793529 non-null float64
mta_tax                  793529 non-null float64
tip_amount               793529 non-null float64
tolls_amount             793529 non-null float64
ehail_fee                0 non-null float64
improvement_surcharge    793529 non-null float64
total_amount             793529 non-null float64
payment_type             793529 non-null int64
trip_type                79

In [26]:
df.store_and_fwd_flag.unique(), df.store_and_fwd_flag.value_counts()

(array(['N', 'Y'], dtype=object), N    792170
 Y      1359
 Name: store_and_fwd_flag, dtype: int64)

In [27]:
df.groupby('VendorID').count().head()

,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
VendorID,,,,,,,,,,,,,,,,,,
1,137694,137694,137694,137694,137694,137694,137694,137694,137694,137694,137694,137694,137694,0,137694,137694,137694,137691
2,655835,655835,655835,655835,655835,655835,655835,655835,655835,655835,655835,655835,655835,0,655835,655835,655835,655835


In [28]:
df.VendorID.value_counts()

2    655835
1    137694
Name: VendorID, dtype: int64

In [29]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-01-01 00:18:50,2018-01-01 00:24:39,N,1,236,236,5,0.70,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1.0
1,2,2018-01-01 00:30:26,2018-01-01 00:46:42,N,1,43,42,5,3.50,14.5,0.5,0.5,0.0,0.0,NaN,0.3,15.8,2,1.0
2,2,2018-01-01 00:07:25,2018-01-01 00:19:45,N,1,74,152,1,2.14,10.0,0.5,0.5,0.0,0.0,NaN,0.3,11.3,2,1.0
3,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,-3.0,-0.5,-0.5,0.0,0.0,NaN,-0.3,-4.3,3,1.0
4,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1.0


In [30]:
from datetime import datetime

In [32]:
#df['pickup_time'] = df['tpep_pickup_datetime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S')) # for yellow
df['pickup_time'] = df['lpep_pickup_datetime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S')) # for green

In [33]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_time
0,2,2018-01-01 00:18:50,2018-01-01 00:24:39,N,1,236,236,5,0.70,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1.0,2018-01-01 00:18:50
1,2,2018-01-01 00:30:26,2018-01-01 00:46:42,N,1,43,42,5,3.50,14.5,0.5,0.5,0.0,0.0,NaN,0.3,15.8,2,1.0,2018-01-01 00:30:26
2,2,2018-01-01 00:07:25,2018-01-01 00:19:45,N,1,74,152,1,2.14,10.0,0.5,0.5,0.0,0.0,NaN,0.3,11.3,2,1.0,2018-01-01 00:07:25
3,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,-3.0,-0.5,-0.5,0.0,0.0,NaN,-0.3,-4.3,3,1.0,2018-01-01 00:32:40
4,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1.0,2018-01-01 00:32:40


saperate date and see if both dates for pick and srop are same, if same, delete one
saperate hrs and min col, count rides with each hr and plot a histogram